In [12]:
# clear all the variables
%reset -f

import os
import IPython as ip
import logging

logger = logging.getLogger('scrapy-tutorial')
logger.setLevel(logging.INFO)

In [8]:
# import the relevant libraries
import scrapy
import re

from bs4 import BeautifulSoup
from scrapy.http import FormRequest, Request
from scrapy.crawler import CrawlerProcess 

# for debugging, uncomment the following line
#from IPython.core.debugger import Tracer as tr
# to call trace, tr()()

In [9]:
# the connect spider definition
class loginSpider(scrapy.Spider):
    name = 'github_login'
    start_urls = ['https://github.com/login']
    login = 'login'
    pwd = 'pwd'
    result = []
    
    def parse(self, response):
        return [FormRequest.from_response(response, 
                                          formdata = {'login': self.login,   'password': self.pwd}, 
                                          callback = self.after_login)]
    
    def after_login(self, response):
        if 'authentication failed' in response.body:
            logging.error('Login failed')
            pass
        else:
            logging.info('Login succeed')
            return Request('https://github.com/' + self.login + '?tab=repositories', 
                       callback=self.parse_repo)
            
    def parse_repo(self, response):
        
        #self.debugPrint(response.body)
        # get the repo list
        soup = BeautifulSoup(response.text, 'lxml')
        list = soup.find_all('li', {'class' : 'col-12 d-block width-full py-4 border-bottom public source'})
        list = [l.find_all('div', {'class' : 'd-inline-block mb-1'})[0] for l in list]
        list = [l.find_all('h3')[0] for l in list]
        list = [l.find_all('a')[0].text for l in list]

        [loginSpider.result.append(re.sub('\s+',' ',l).encode('utf-8')) for l in list]
        
    def debugPrint(self, message):
        with open(r'C:\Temp\debug.txt', 'a') as f:
            f.write(message)
            f.close()

In [ ]:
# set the proxy
os.environ["http_proxy"] = "http://ip:port"
os.environ["https_proxy"] = "https://ip:port"

# set the logging policy
logger = logging.getLogger('scrapy-tutorial')
logger.setLevel(logging.INFO)

process = CrawlerProcess({
'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

sp = loginSpider()
process.crawl(sp)
process.start(stop_after_crawl=True)

print('repo found: {0}'.format(loginSpider.result))

In [12]:
# terminate the kernel
app = ip.Application.instance()
app.kernel.do_shutdown(restart=False)

{'restart': False, 'status': 'ok'}